<head><b><center>CIENCIA DE DATOS</center>
<center>Entregables para el Despliegue</center></b>
    

<center>Profesor: Gabriel Jara </center></head><br>
El presente Jupyter Notebook busca abordar el despliegue de productos de la ciencia de datos, en la forma de: pipeline de pre-procesamiento y modelo predictivo listo para la explotación. 

<b>Pipeline de pre-procesamiento: </b>

Organizamos nuestro pre-procesamiento como una serie de transformaciones, las que luego utilizamos en secuencia usando un <b>Pipeline</b> de <i>scikit-learn</i>. Este enfoque nos permite encadenar pasos como la eliminación de columnas, codificación de variables categóricas y limpieza de datos de manera estructurada y reutilizable. Al definir el pipeline, aseguramos que las mismas transformaciones se apliquen de forma consistente tanto a los datos de entrenamiento como a los de prueba o producción.

Para guardar el pipeline, así como también algunos modelos, usaremos <b>joblib</b>, biblioteca de Python diseñada para guardar y cargar objetos complejos de manera eficiente, especialmente utilizada en proyectos de ciencia de datos y machine learning.

In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib

# Creamos un transformador para eliminar columnas específicas
class EliminarColumnas(BaseEstimator, TransformerMixin):
    def __init__(self, columnas):
        self.columnas = columnas

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Eliminamos las columnas indicadas
        return X.drop(columns=self.columnas)

# Creamos un transformador para mapear el sexo a números
class CodificarSexo(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Convertimos 'Sex' a 0 y 1
        X = X.copy()
        X['Sex'] = X['Sex'].map({'male': 0, 'female': 1})
        return X

# Creamos un transformador para eliminar filas con valores faltantes
class EliminarNulos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Eliminamos filas con nulos
        return X.dropna()

# Definimos el pipeline de procesamiento
pipeline_titanic = Pipeline([
    ('eliminar_columnas', EliminarColumnas(['Name', 'Ticket', 'Cabin', 'Embarked'])),
    ('codificar_sexo', CodificarSexo()),
    ('eliminar_nulos', EliminarNulos())
])

# Guardamos el pipeline para uso futuro
joblib.dump(pipeline_titanic, 'pipeline_titanic.pkl')

['pipeline_titanic.pkl']

Hemos guardado el Pipeline, luego podemos cargarlo a otro entorno y así utilizarlo para obtener la misma transformación sobre los datos. Eso sí, es necesario también volver a definir las transformaciones, por lo cual lo más simple es importarlas desde un módulo propio. 

Así, podemos ejecutar la siguiente celda de forma independiente de la anterior. Puede probar reiniciando el kernel. 

In [4]:
import pandas as pd
import numpy as np
import joblib

# Necesitamos recuperar los transformadores que usa el pipeline
from transformadores_titanic import EliminarColumnas, CodificarSexo, EliminarNulos

# Recuperamos también el pipeline 
pipeline_titanic = joblib.load('pipeline_titanic.pkl')

# Función para cargar los datos
def cargar_datos(path):
    return pd.read_csv(path)

# Aplicamos el pipeline al DataFrame con los datos del Titanic
df = cargar_datos("Titanic.csv")
df_procesado = pipeline_titanic.fit_transform(df)

# Convertimos a arreglo NumPy
datos = df_procesado.to_numpy()

# Separamos X e y
y = datos[:,1]
X = datos[:,2:]
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

ModuleNotFoundError: No module named 'transformadores_titanic'

<b>Guardando Modelos sklearn: </b>

Partamos buscando un buen clasificador de tipo árbol, y guardemoslo usando joblib.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
import joblib

# En esta celda sólo vamos a entrenar, no usaremos testeo
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=123)

# Definimos el modelo de clasificación
clf = DecisionTreeClassifier()

# Definimos la grilla de parámetros que vamos a probar
param_grid = {'max_depth': [1,2,3,4,5,6,7,8,9,10,11,12,None],
              'criterion': ['gini','entropy'],
              'min_samples_split': [2,3,4,5,6,7,8],
              'min_samples_leaf': [1,2,3,4,5,6],
              'ccp_alpha': [0,0.0001,0.001,0.01,0.1,1]
             }

# Realizamos búsqueda de grilla con validación cruzada de 5 carpetas
grid_search = GridSearchCV(clf, param_grid, cv=5)

# Ajustar el modelo con búsqueda de grilla
grid_search.fit(X_train, y_train)

# Imprimimos mejores parámetros y mejor puntaje
print("Mejores parámetros: ", grid_search.best_params_)
print("Mejor puntaje: ", grid_search.best_score_)

# Volvemos a entrenar el modelo, con mejores parámetros
clf = DecisionTreeClassifier(max_depth= grid_search.best_params_['max_depth'],
                             criterion= grid_search.best_params_['criterion'],
                             min_samples_split= grid_search.best_params_['min_samples_split'],
                             min_samples_leaf= grid_search.best_params_['min_samples_leaf'],
                             ccp_alpha=grid_search.best_params_['ccp_alpha'])
model = clf.fit(X_train, y_train)

# Guardamos el modelo entrenado, listo para usar más adelante.
joblib.dump(model, "modelo_titanic_tree.pkl")


Hemos guardado el modelo como archivo <b>pkl</b> (abreviación de pickle), forma de serializar objetos de Python, es decir, convertirlos a un formato binario que puede guardarse en un archivo y luego reconstruirse exactamente igual. 

La siguiente celda usa los recursos previamente guardados, tanto pipeline como modelo de clasificación. Por lo mismo, se puede interrumpir el kernel del jupytr y retomar desde la siguiente celda, sin problema. El modelo se cargará ya entrenado, listo para usar en nuevas predicciones. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
import joblib
from transformadores_titanic import EliminarColumnas, CodificarSexo, EliminarNulos

# Cargamos nuevos datos y los preprocesamos con el pipeline
pipeline = joblib.load("pipeline_titanic.pkl")
df_nuevo = pd.read_csv("Titanic.csv")
df_procesado = pipeline.transform(df_nuevo)

# Convertimos a arreglo NumPy
datos = df_procesado.to_numpy()

# Separamos X e y
y = datos[:,1]
X = datos[:,2:]
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

# Esta vez sólo usaremos los datos de testeo, no necesitamos entrenamiento.
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Cargamos el modelo
modelo = joblib.load("modelo_titanic_tree.pkl")

# ¿Cómo sabemos qué modelo tenemos guardado?
# Podemos consultarlo directo al propio modelo
print('TIPO DE MODELO:',type(modelo))
print('HIPERPARÁMETROS',modelo.get_params())

# Reportamos resultados predictivos sobre la muestra de testeo
y_hat = modelo.predict(X_test)
print('La exactitud de testeo del modelo guardado es:',modelo.score(X_test,y_test))

<b>Guardemos una ANN de Keras:</b>

Keras no está diseñado para trabajar con joblib, pero por otro lado no es necesario dado que tiene sus propios métodos para almacenar y cargar modelos. En la siguiente celda creamos, entrenamos y guardamos una red neuronal. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import joblib
from transformadores_titanic import EliminarColumnas, CodificarSexo, EliminarNulos

# Cargamos nuevos datos y los preprocesamos con el pipeline
pipeline = joblib.load("pipeline_titanic.pkl")
df_nuevo = pd.read_csv("Titanic.csv")
df_procesado = pipeline.transform(df_nuevo)

# Convertimos a arreglo NumPy
datos = df_procesado.to_numpy()

# Separamos X e y
y = datos[:,1]
X = datos[:,2:]
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

# Esta vez sólo usaremos los datos de entrenamiento, no necesitamos testeo.
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=123)

# Modelo de Red Neuronal, nada especial
model = Sequential()
entrada_dim = len(X_train[0])
model.add(Dense(units=32, activation='relu', input_dim=entrada_dim))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamos el modelo con los datos de entrenamiento
model.fit(X_train, y_train, epochs=40, batch_size=1, verbose = 0)

# Guardamos el modelo ya entrenado, para usarlo más adelante
model.save("modelo_titanic.keras")

La siguiente celda se puede ejecutar independiente de lo anterior, tanto el preprocesamiento como el modelo se recuperan como objetos previamente guardados. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import load_model
import joblib
from transformadores_titanic import EliminarColumnas, CodificarSexo, EliminarNulos

# Cargamos nuevos datos y los preprocesamos con el pipeline
pipeline = joblib.load("pipeline_titanic.pkl")
df_nuevo = pd.read_csv("Titanic.csv")
df_procesado = pipeline.transform(df_nuevo)

# Convertimos a arreglo NumPy
datos = df_procesado.to_numpy()

# Separamos X e y
y = datos[:,1]
X = datos[:,2:]
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

# Esta vez sólo usaremos los datos de testeo, no necesitamos entrenamiento.
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Recuperamos el modelo
modelo = load_model("modelo_titanic.keras")

# Averiguemos qué modelo es este
modelo.summary()

# Evaluamos el modelo con datos de testeo
loss, accuracy = modelo.evaluate(X_test, y_test)

ModuleNotFoundError: No module named 'transformadores_titanic'

<b>Conclusión:</b>

Hemos encapsulado el preprocesamiento y los modelos en productos entregables, que podemos cargar a nuevos ambientes para poder seguir utilizándolo. Si bien nuestro pipeline sólo consideró el pre-procesamiento, podríamos incluir desde la captura de datos hasta la generación y entrenamiento del modelo predictivo, todo dentro del mismo pipeline. 

Esto resulta fundamental para la eventual integración de nuestros modelos predictivos en sistemas de información que los pongan disponibles para la operación, facilitando una integración confiable, limpia y escalable de los modelos en producción.